# Voting

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

f_size = 11
# Importing the dataset
# dataset = pd.read_csv('CVD dataset-plin-7-9.csv') #14
dataset = pd.read_csv('CVD-dataset-plin-0-4-6-7-9.csv') #11
# dataset = pd.read_csv('CVD dataset2.csv')
# dataset = pd.read_csv('CVD-dataset-2-3-11-13-15.csv')
# dataset = pd.read_csv('CVD dataset-plin-4-7-9.csv') #διαβάζω το dataset που δεν περιλαμβανει τα features 7 και 9
# dataset = pd.read_csv('CVD-dataset-1-2-3-5-10-11-12-13-14-15.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, f_size].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
12


In [2]:
dataNoCvd = data[~(data[:,f_size] > 0.0)]
dataCvd = data[~(data[:,f_size] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [3]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
#     print("len 1-2:",len(temp1),len(temp2))
    temp3 = np.delete(temp1, slice(0, 46), axis=0)
    temp4 = np.delete(temp2, slice(0, 4), axis=0)
#     print("len 3-4:",len(temp3),len(temp4))
    temp5 = np.concatenate((temp3, temp4))
#     print(len(temp5))
    return temp3, temp4, temp5

In [4]:
def create_train_test_sets(f_size,cv,dataNoCvd,dataCvd,val_ratio):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    noCvd_val = round(504*val_ratio)-Cvd_val
#     print(noCvd_val)
    
    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
        X_val = []
        y_val = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)
#         print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd)        
        
        if i!=9:
        #όταν δεν είμαστε στο 10ο fold(ξεκινάμε από 0) παίρνουμε τα επόμενα 56 και 4 instaces για το validation set
            validationSubset_total = find_testValSubset(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val,dataNoCvd,dataCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val,dataNoCvd,dataCvd)
        else:
        #όταν είμαστε στο 10ο fold παίρνουμε τα προηγούμενα 56 και 4 instaces για το validation set
#         DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd
            validationSubset_total = find_testValSubset(0,noCvd_val,0,Cvd_val,dataNoCvd,dataCvd)
            print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd)
    
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, f_size, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, f_size), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
        
        X_val.append(np.delete(validationSubset_total, f_size, axis=1))
        y_val_temp = np.delete(validationSubset_total, slice(0, f_size), axis=1)
        y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
        X_val_temp = np.array(X_val)
        X_val = X_val_temp[0]
        y_val_temp = np.array(y_val)
        y_val = y_val_temp[0]
        
        X_train.append(np.delete(trainSubset_total, f_size, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, f_size), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        validation_total.append([X_val,y_val])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd

In [5]:
train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(f_size,10,dataNoCvd,dataCvd,0.1)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9
468 519 36 41


In [6]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)
    ratio = find_ratio(validation_total[i])
    print("ratio in validation set:", ratio)

454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in vali

In [61]:
from xgboost import XGBClassifier


def my_fit(ratio,dataNoCvd,dataCvd, validation_total, params):   
    Subarray=[]
    models = []
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0


    for i in range(numOfSubsamples):
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=params[8], gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=params[9], reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=params[10], verbosity=1)
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], f_size, axis=1)
        y=np.delete(Subarray[i], slice(0, f_size), axis=1)
        y=np.reshape(y, len(y))
        
        X_val = validation_total[i][0]
        y_val = validation_total[i][1]
        
        eval_set  = [(X,y), (X_val,y_val)]
        
        classifier.fit( X, y, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False)
#         classifier.fit( X, y)
        
        models.append(classifier)
    return models 

In [8]:
def predict_weighted_average(models,X,weights):
#     print("efwef",len(weights), len(models))
    y_pred = []
    y_pred_models = []
#     weights = [0.16, 0.175, 0.2, 0.147, 0.148, 0.16]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
#     print(models_size)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
#             print(i)
#             print(y_pred_models[i][j])
#             print(weights[i])
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred        

In [9]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params, weights):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i], validation_total,params)


        y_pred = predict_weighted_average(models,X_test,weights)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [10]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bylevel")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")
    list9 = params.get("colsample_bytree")
    list10 = params.get("reg_alpha")
    list11 = params.get("subsample")
    

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p,q,r,s] for i in list1  
                                          for j in list2 
                                          for k in list3 
                                          for l in list4 
                                          for m in list5 
                                          for n in list6 
                                          for o in list7
                                          for p in list8
                                          for q in list9
                                          for r in list10
                                          for s in list11] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [11]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [12]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params,weights):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
#     print(len(combs))
    for j in range(len(combs)):
#         print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j],weights)
        if mean(auc)>0.69:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print (style.BOLD + 'weights     ' + style.END, weights)
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
#         else:
#             print(combs[j])
#             print("accuracy    ",mean(accuracy))
#             print("specificity ",mean(specificity))
#             print("sensitivity ",mean(sensitivity))
#             print("auc         ",mean(auc))
#             print("weights     ",weights)
            
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc), weights])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

 Αποτέλεσμα της gridsearch με τις ακόλουθες παραμέτρους, γίνονται print όλα και αυτά που ικανοποιούν την συνθήκη : if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5 είναι με bold και μεταξύ γραμμών.

In [33]:
#σωστό
params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1 0.75
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4],
                    "colsample_bytree" :[0.75],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
            
        }
# parameters
# [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0, 0.5, 1, 0]
# accuracy     0.6089285714285715
# specificity  0.5937782805429864
# sensitivity  0.81
# auc          0.7018891402714932

In [35]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6124999999999999
specificity  0.5995098039215686
sensitivity  0.785
auc          0.6922549019607843
weights      [0.05, 0.05, 0.2, 0.05, 0.35, 0.3]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285714
specificity  0.5956636500754148
sensitivity  0.785
auc          0.6903318250377074
weights      [0.05, 0.05, 0.2, 0.1, 0.35, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5875
specificity  0.5706259426847662
sensitivity  0.81
auc          0.6903129713423831
weights      [0.05, 0.05, 0.2, 0.3, 0.15, 0.25]
-------------------------------------------------------------

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5910714285714286
specificity  0.5745475113122172
sensitivity  0.81
auc          0.6922737556561086
weights      [0.1, 0.2, 0.05, 0.2, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5928571428571429
specificity  0.5764705882352941
sensitivity  0.81
auc          0.6932352941176471
weights      [0.1, 0.2, 0.05, 0.25, 0.15, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5875
specificity  0.5706636500754148
sensitivity  0.81
auc          0.6903318250377074
weights      [0.1, 0.2, 0.05, 0.25, 0.2, 0.2]
----------------------------------------------------------------
--

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6
specificity  0.5841628959276018
sensitivity  0.81
auc          0.6970814479638009
weights      [0.15, 0.1, 0.25, 0.2, 0.1, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.5982142857142857
specificity  0.5822398190045249
sensitivity  0.81
auc          0.6961199095022624
weights      [0.15, 0.15, 0.15, 0.15, 0.15, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285715
specificity  0.5937782805429864
sensitivity  0.81
auc          0.7018891402714932
weights      [0.15, 0.15, 0.2, 0.15, 0.15, 0.2]
----------------------------------------------------------------
-

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6125
specificity  0.5995098039215686
sensitivity  0.785
auc          0.6922549019607843
weights      [0.2, 0.1, 0.15, 0.1, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285714
specificity  0.5956636500754148
sensitivity  0.785
auc          0.6903318250377074
weights      [0.2, 0.1, 0.2, 0.05, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6089285714285714
specificity  0.5956636500754148
sensitivity  0.79
auc          0.6928318250377075
weights      [0.2, 0.1, 0.2, 0.1, 0.15, 0.25]
----------------------------------------------------------------
--

----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6178571428571429
specificity  0.6052790346907994
sensitivity  0.785
auc          0.6951395173453997
weights      [0.25, 0.1, 0.15, 0.05, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6125
specificity  0.5995098039215686
sensitivity  0.785
auc          0.6922549019607843
weights      [0.25, 0.1, 0.15, 0.1, 0.15, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy     0.6125
specificity  0.5995098039215686
sensitivity  0.79
auc          0.6947549019607844
weights      [0.25, 0.1, 0.2, 0.05, 0.15, 0.25]
----------------------------------------------------------------
--------

In [36]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6089285714285715
spes     : 0.5937782805429864
sens     : 0.81
auc      : 0.7018891402714932
weights  : [0.15, 0.15, 0.2, 0.15, 0.15, 0.2]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6089285714285714
spes     : 0.5937782805429864
sens     : 0.81
auc      : 0.7018891402714933
weights  : [0.2, 0.05, 0.3, 0.1, 0.15, 0.2]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6107142857142857
spes     : 0.5957013574660633
sens     : 0.81
auc      : 0.7028506787330318
weights  : [0.2, 0.1, 0.25, 0.1, 0.15, 0.2]
------------------------------------------------
parameters : [0.3, 2, 1000, 3, 0, 0.5, 1, 4, 0.75, 0, 1.0]
accuracy : 0.6071428571428571
spes     : 0.5918552036199095
sens     : 0.81
auc      : 0.7009276018099547
weights  : [0.2, 0.15, 0.15, 0.1, 0.15, 0.25]
-------------

In [49]:
#σωστό
params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
# accuracy : 0.6589285714285714
# spes     : 0.6532051282051282
# sens     : 0.74
# auc      : 0.6966025641025642
# [13, 11, 15, 2, 3]
# [13, 11, 15, 2, 3, 14, 1, 12, 10, 5, 6, 4, 8, 9, 0, 7, 16]

In [50]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6642857142857143
specificity  0.6610482654600301
sensitivity  0.72
auc          0.6905241327300151
weights      [0.05, 0.1, 0.15, 0.25, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6660714285714285
specificity  0.6629336349924585
sensitivity  0.72
auc          0.6914668174962293
weights      [0.05, 0.1, 0.2, 0.25, 0.25, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6660714285714285
specificity  0.6629336349924585
sensitivity  0.72
auc          0.6914668174962293
weights      [0.05, 0.1, 0.25, 0.25, 0.25, 0.1]
------------------------------------------------------

----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6767857142857143
specificity  0.6745098039215687
sensitivity  0.72
auc          0.6972549019607843
weights      [0.1, 0.15, 0.2, 0.2, 0.25, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6660714285714285
specificity  0.6629713423831071
sensitivity  0.72
auc          0.6914856711915536
weights      [0.1, 0.15, 0.2, 0.25, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy     0.6732142857142857
specificity  0.6705882352941177
sensitivity  0.72
auc          0.6952941176470588
weights      [0.1, 0.15, 0.25, 0.2, 0.25, 0.05]
--------------------------------------------------------

In [54]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.697:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6767857142857143
spes     : 0.6745098039215687
sens     : 0.72
auc      : 0.6972549019607843
weights  : [0.1, 0.15, 0.2, 0.2, 0.25, 0.1]
------------------------------------------------
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6642857142857143
spes     : 0.6590874811463047
sens     : 0.74
auc      : 0.6995437405731524
weights  : [0.1, 0.3, 0.05, 0.05, 0.25, 0.25]
------------------------------------------------
2


In [55]:
#σωστό
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.1],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
# accuracy : 0.6482142857142857
# spes     : 0.6417043740573152
# sens     : 0.74
# auc      : 0.6908521870286576
# [13, 15, 11, 2, 3]
# [13, 15, 11, 2, 3, 1, 14, 12, 10, 5, 6, 8, 4, 9, 0, 7, 16]

In [56]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6417043740573152
sensitivity  0.745
auc          0.6933521870286576
weights      [0.15, 0.1, 0.4, 0.1, 0.2, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571429
specificity  0.635972850678733
sensitivity  0.745
auc          0.6904864253393665
weights      [0.15, 0.15, 0.35, 0.15, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.6360482654600301
sensitivity  0.745
auc          0.6905241327300151
weights      [0.15, 0.2, 0.15, 0.2, 0.15, 0.15]
----------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6464285714285715
specificity  0.6398567119155355
sensitivity  0.745
auc          0.6924283559577677
weights      [0.2, 0.2, 0.15, 0.15, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6446428571428572
specificity  0.6379336349924585
sensitivity  0.745
auc          0.6914668174962293
weights      [0.2, 0.2, 0.15, 0.2, 0.1, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6417797888386124
sensitivity  0.745
auc          0.6933898944193062
weights      [0.2, 0.2, 0.15, 0.2, 0.15, 0.1]
-------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6446428571428572
specificity  0.6378959276018099
sensitivity  0.745
auc          0.691447963800905
weights      [0.2, 0.25, 0.15, 0.2, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6464285714285715
specificity  0.6398190045248868
sensitivity  0.745
auc          0.6924095022624435
weights      [0.2, 0.25, 0.15, 0.25, 0.1, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6517857142857143
specificity  0.6456259426847663
sensitivity  0.745
auc          0.6953129713423831
weights      [0.2, 0.25, 0.2, 0.1, 0.1, 0.15]
------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.6360859728506787
sensitivity  0.745
auc          0.6905429864253394
weights      [0.25, 0.1, 0.4, 0.05, 0.1, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6418174962292609
sensitivity  0.745
auc          0.6934087481146305
weights      [0.25, 0.1, 0.4, 0.05, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.6360482654600301
sensitivity  0.745
auc          0.6905241327300151
weights      [0.25, 0.1, 0.4, 0.1, 0.05, 0.1]
-------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.6360482654600301
sensitivity  0.745
auc          0.6905241327300151
weights      [0.25, 0.2, 0.1, 0.2, 0.1, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571428
specificity  0.635972850678733
sensitivity  0.745
auc          0.6904864253393665
weights      [0.25, 0.2, 0.1, 0.25, 0.1, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6398944193061841
sensitivity  0.77
auc          0.704947209653092
weights      [0.25, 0.2, 0.15, 0.1, 0.1, 0.2]
------------------------------------------------

...

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6392857142857142
specificity  0.6302790346907994
sensitivity  0.77
auc          0.7001395173453997
weights      [0.3, 0.15, 0.1, 0.1, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571429
specificity  0.6360105580693816
sensitivity  0.745
auc          0.6905052790346907
weights      [0.3, 0.15, 0.1, 0.15, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6410714285714286
specificity  0.6321644042232277
sensitivity  0.77
auc          0.7010822021116139
weights      [0.3, 0.15, 0.15, 0.1, 0.05, 0.25]
-------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6464285714285715
specificity  0.6398567119155354
sensitivity  0.745
auc          0.6924283559577677
weights      [0.3, 0.2, 0.15, 0.15, 0.05, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571429
specificity  0.6360105580693816
sensitivity  0.745
auc          0.6905052790346908
weights      [0.3, 0.2, 0.15, 0.15, 0.1, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6410714285714285
specificity  0.6321644042232277
sensitivity  0.77
auc          0.7010822021116139
weights      [0.3, 0.2, 0.2, 0.05, 0.15, 0.1]
--------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6357142857142857
specificity  0.6263574660633484
sensitivity  0.77
auc          0.6981787330316742
weights      [0.35, 0.05, 0.15, 0.1, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6339285714285714
specificity  0.6244343891402715
sensitivity  0.77
auc          0.6972171945701358
weights      [0.35, 0.05, 0.15, 0.15, 0.1, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6339285714285714
specificity  0.6244343891402715
sensitivity  0.77
auc          0.6972171945701358
weights      [0.35, 0.05, 0.15, 0.15, 0.15, 0.15]
----------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571429
specificity  0.635972850678733
sensitivity  0.745
auc          0.6904864253393664
weights      [0.35, 0.15, 0.15, 0.15, 0.15, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6428571428571429
specificity  0.635972850678733
sensitivity  0.745
auc          0.6904864253393664
weights      [0.35, 0.15, 0.15, 0.2, 0.1, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6410714285714285
specificity  0.6321266968325792
sensitivity  0.77
auc          0.7010633484162896
weights      [0.35, 0.15, 0.2, 0.05, 0.15, 0.1]
----------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6464285714285715
specificity  0.6398190045248869
sensitivity  0.745
auc          0.6924095022624435
weights      [0.4, 0.05, 0.35, 0.1, 0.05, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6357142857142857
specificity  0.6263197586726998
sensitivity  0.77
auc          0.6981598793363499
weights      [0.4, 0.1, 0.15, 0.1, 0.2, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6339285714285714
specificity  0.6244343891402715
sensitivity  0.77
auc          0.6972171945701358
weights      [0.4, 0.1, 0.15, 0.15, 0.1, 0.1]
---------------------------------------------

In [57]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6517857142857143
spes     : 0.6437028657616893
sens     : 0.77
auc      : 0.7068514328808446
weights  : [0.2, 0.25, 0.2, 0.1, 0.15, 0.1]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6392857142857142
spes     : 0.6302413273001508
sens     : 0.77
auc      : 0.7001206636500754
weights  : [0.25, 0.1, 0.2, 0.15, 0.1, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6482142857142857
spes     : 0.6398567119155354
sens     : 0.77
auc      : 0.7049283559577677
weights  : [0.25, 0.15, 0.2, 0.1, 0.1, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6482142857142857
spes     : 0.6398944193061841
sens     : 0.77
auc      : 0.704947209653092
weights  : [0.25, 0.2, 0.15, 0.1, 0.1, 0.2]
------

In [58]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.705:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6517857142857143
spes     : 0.6437028657616893
sens     : 0.77
auc      : 0.7068514328808446
weights  : [0.2, 0.25, 0.2, 0.1, 0.15, 0.1]
------------------------------------------------
parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.65
spes     : 0.6417797888386123
sens     : 0.77
auc      : 0.7058898944193062
weights  : [0.25, 0.2, 0.2, 0.1, 0.1, 0.15]
------------------------------------------------
2


In [40]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.2],
                    "subsample"        :[1.0]
    
        }
# [15, 13, 14, 11, 3, 1, 2, 10, 12, 5]
# [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
# 0.6446428571428572
# 0.6360105580693816
# 0.76
# 0.6980052790346908

In [41]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

In [43]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.67:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy : 0.6428571428571428
spes     : 0.637971342383107
sens     : 0.71
auc      : 0.6739856711915535
weights  : [0.25, 0.2, 0.05, 0.05, 0.05, 0.4]
------------------------------------------------
parameters : [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy : 0.6428571428571429
spes     : 0.6380090497737556
sens     : 0.71
auc      : 0.6740045248868778
weights  : [0.3, 0.15, 0.05, 0.05, 0.05, 0.4]
------------------------------------------------
parameters : [0.4, 3, 1000, 2, 0.5, 0.5, 1, 3, 0.5, 0.2, 1.0]
accuracy : 0.6375
spes     : 0.6322021116138763
sens     : 0.71
auc      : 0.6711010558069381
weights  : [0.35, 0.15, 0.05, 0.05, 0.05, 0.35]
------------------------------------------------
3


In [78]:
#σωστό
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
# accuracy : 0.6285714285714286
# spes     : 0.616817496229261
# sens     : 0.785
# auc      : 0.7009087481146304
# [14, 15, 3, 13, 11, 1, 2, 12, 5, 10, 8]
# [14, 15, 3, 13, 11, 1, 2, 12, 5, 10, 8, 0, 4, 6, 7, 9, 16]
# 14-15-3-13-11-1-2-12-5-10-8
# plin-0-4-6-7-9

In [79]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6053571428571428
specificity  0.5899321266968326
sensitivity  0.81
auc          0.6999660633484163
weights      [0.05, 0.05, 0.15, 0.15, 0.25, 0.35]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6125
specificity  0.5995475113122172
sensitivity  0.785
auc          0.6922737556561086
weights      [0.05, 0.05, 0.15, 0.25, 0.3, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6053571428571428
specificity  0.5899698340874812
sensitivity  0.81
auc          0.6999849170437406
weights      [0.05, 0.05, 0.2, 0.1, 0.2, 0.4]
-----------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6160714285714286
specificity  0.6015082956259427
sensitivity  0.81
auc          0.7057541478129713
weights      [0.05, 0.05, 0.3, 0.15, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6196428571428572
specificity  0.6072775263951734
sensitivity  0.785
auc          0.6961387631975867
weights      [0.05, 0.05, 0.3, 0.15, 0.25, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6107142857142857
specificity  0.5975867269984917
sensitivity  0.785
auc          0.6912933634992459
weights      [0.05, 0.05, 0.3, 0.2, 0.25, 0.15]
----------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6107142857142858
specificity  0.5976244343891403
sensitivity  0.785
auc          0.6913122171945701
weights      [0.05, 0.1, 0.1, 0.3, 0.3, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6
specificity  0.5842006033182504
sensitivity  0.81
auc          0.6971003016591252
weights      [0.05, 0.1, 0.15, 0.15, 0.15, 0.4]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6053571428571428
specificity  0.5899698340874812
sensitivity  0.81
auc          0.6999849170437406
weights      [0.05, 0.1, 0.15, 0.15, 0.2, 0.35]
----------------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6125
specificity  0.5976244343891403
sensitivity  0.81
auc          0.7038122171945701
weights      [0.05, 0.1, 0.25, 0.1, 0.2, 0.3]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6178571428571429
specificity  0.6034313725490196
sensitivity  0.81
auc          0.7067156862745098
weights      [0.05, 0.1, 0.25, 0.1, 0.25, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6107142857142858
specificity  0.5957390648567119
sensitivity  0.81
auc          0.7028695324283559
weights      [0.05, 0.1, 0.25, 0.15, 0.2, 0.25]
--------------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6285714285714286
specificity  0.6168552036199095
sensitivity  0.785
auc          0.7009276018099547
weights      [0.05, 0.1, 0.3, 0.25, 0.25, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6142857142857143
specificity  0.6014705882352941
sensitivity  0.785
auc          0.6932352941176471
weights      [0.05, 0.1, 0.35, 0.05, 0.1, 0.35]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6160714285714286
specificity  0.6033559577677224
sensitivity  0.785
auc          0.6941779788838612
weights      [0.05, 0.1, 0.35, 0.05, 0.15, 0.3]
---------------------------------------------

...

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6375000000000001
specificity  0.6265082956259427
sensitivity  0.785
auc          0.7057541478129713
weights      [0.2, 0.1, 0.25, 0.05, 0.15, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6339285714285714
specificity  0.6245098039215686
sensitivity  0.76
auc          0.6922549019607843
weights      [0.2, 0.1, 0.25, 0.05, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6357142857142857
specificity  0.6245475113122172
sensitivity  0.785
auc          0.7047737556561086
weights      [0.2, 0.1, 0.25, 0.1, 0.1, 0.25]
--------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6232142857142857
specificity  0.6109728506787331
sensitivity  0.785
auc          0.6979864253393665
weights      [0.2, 0.15, 0.1, 0.1, 0.15, 0.3]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6160714285714286
specificity  0.6013574660633484
sensitivity  0.81
auc          0.7056787330316742
weights      [0.2, 0.15, 0.1, 0.1, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6178571428571429
specificity  0.6052413273001508
sensitivity  0.785
auc          0.6951206636500754
weights      [0.2, 0.15, 0.1, 0.15, 0.15, 0.25]
--------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6303571428571428
specificity  0.6206259426847662
sensitivity  0.76
auc          0.6903129713423831
weights      [0.2, 0.15, 0.3, 0.05, 0.05, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6375
specificity  0.626395173453997
sensitivity  0.785
auc          0.7056975867269984
weights      [0.2, 0.15, 0.3, 0.05, 0.1, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6464285714285715
specificity  0.6380090497737556
sensitivity  0.76
auc          0.6990045248868778
weights      [0.2, 0.15, 0.3, 0.05, 0.15, 0.15]
--------------------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6232142857142857
specificity  0.6109351432880845
sensitivity  0.785
auc          0.6979675716440422
weights      [0.2, 0.2, 0.15, 0.2, 0.15, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6303571428571428
specificity  0.6206259426847662
sensitivity  0.76
auc          0.6903129713423831
weights      [0.2, 0.2, 0.15, 0.2, 0.2, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6142857142857143
specificity  0.6013574660633484
sensitivity  0.785
auc          0.6931787330316742
weights      [0.2, 0.2, 0.15, 0.25, 0.15, 0.05]
--------------------------------------------------

----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6089285714285715
specificity  0.5955505279034691
sensitivity  0.785
auc          0.6902752639517346
weights      [0.2, 0.3, 0.05, 0.1, 0.2, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6214285714285714
specificity  0.6090497737556562
sensitivity  0.785
auc          0.697024886877828
weights      [0.2, 0.3, 0.15, 0.05, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy     0.6196428571428572
specificity  0.6072021116138763
sensitivity  0.785
auc          0.6961010558069382
weights      [0.25, 0.05, 0.05, 0.1, 0.25, 0.3]
------------------------------------------------

In [81]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.71:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6267857142857143
spes     : 0.6129713423831071
sens     : 0.81
auc      : 0.7114856711915535
weights  : [0.05, 0.05, 0.3, 0.25, 0.25, 0.1]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.625
spes     : 0.6110482654600302
sens     : 0.81
auc      : 0.710524132730015
weights  : [0.05, 0.1, 0.35, 0.1, 0.2, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6285714285714286
spes     : 0.6149321266968326
sens     : 0.81
auc      : 0.7124660633484163
weights  : [0.05, 0.1, 0.35, 0.15, 0.2, 0.15]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6303571428571428
spes     : 0.6168552036199095
sens     : 0.81
auc      : 0.7134276018099548
weights  : [0.05, 0.1, 0.4, 0.2, 0.2, 0.05]
-----------------------

In [82]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.72:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6267857142857143
spes     : 0.6110482654600302
sens     : 0.835
auc      : 0.7230241327300151
weights  : [0.05, 0.15, 0.35, 0.2, 0.15, 0.1]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6035714285714285
spes     : 0.5841628959276018
sens     : 0.86
auc      : 0.7220814479638009
weights  : [0.05, 0.2, 0.25, 0.2, 0.15, 0.15]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6089285714285715
spes     : 0.5899321266968326
sens     : 0.86
auc      : 0.7249660633484163
weights  : [0.05, 0.2, 0.3, 0.15, 0.1, 0.2]
------------------------------------------------
parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 2, 3, 0.5, 0, 1.0]
accuracy : 0.6339285714285714
spes     : 0.6187782805429864
sens     : 0.835
auc      : 0.7268891402714932
weights  : [0.05, 0.2, 0.4, 0.15, 0.15, 0.05]
-----

In [90]:
#σωστό
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4],
                    "colsample_bytree" :[0.75],
                    "reg_alpha"        :[0.1],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.4, 3, 1000, 3, 0, 0.5, 1.5, 4, 0.75, 0.1, 0.8]
# accuracy : 0.65
# spes     : 0.6418929110105581
# sens     : 0.76
# auc      : 0.700946455505279
# [14, 3, 11, 13, 1, 15, 2, 12, 5, 10, 4, 0, 6, 8]
# [14, 3, 11, 13, 1, 15, 2, 12, 5, 10, 4, 0, 6, 8, 7, 9, 16]

In [91]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.4, 3, 1000, 3, 0, 0.5, 1.5, 4, 0.75, 0.1, 1.0]
accuracy     0.5946428571428571
specificity  0.5783559577677225
sensitivity  0.81
auc          0.6941779788838612
weights      [0.05, 0.15, 0.25, 0.25, 0.1, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 3, 0, 0.5, 1.5, 4, 0.75, 0.1, 1.0]
accuracy     0.6089285714285715
specificity  0.5956636500754148
sensitivity  0.785
auc          0.6903318250377074
weights      [0.1, 0.05, 0.25, 0.2, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.4, 3, 1000, 3, 0, 0.5, 1.5, 4, 0.75, 0.1, 1.0]
accuracy     0.6089285714285714
specificity  0.5956636500754148
sensitivity  0.785
auc          0.6903318250377074
weights      [0.1, 0.1, 0.2, 0.2, 0.2, 0.2]
-------------------------------------------

In [92]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

0


In [13]:
#σωστό
params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.75],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
# accuracy : 0.6625
# spes     : 0.6572398190045249
# sens     : 0.735
# auc      : 0.6961199095022624
# [3, 14, 13, 1, 11, 2, 15, 12, 5, 10, 8]
# [3, 14, 13, 1, 11, 2, 15, 12, 5, 10, 8, 6, 4, 0, 9, 7, 16]

In [14]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.65
specificity  0.6437405731523379
sensitivity  0.74
auc          0.6918702865761689
weights      [0.05, 0.05, 0.3, 0.2, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6357142857142857
specificity  0.6264328808446455
sensitivity  0.765
auc          0.6957164404223228
weights      [0.05, 0.05, 0.3, 0.3, 0.05, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.65
specificity  0.6437782805429865
sensitivity  0.74
auc          0.6918891402714932
weights      [0.05, 0.05, 0.3, 0.3, 0.15, 0.15]
----------------------------------------------------------------
---------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6553571428571429
specificity  0.6495852187028658
sensitivity  0.735
auc          0.6922926093514329
weights      [0.05, 0.1, 0.25, 0.2, 0.2, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6642857142857143
specificity  0.6592383107088989
sensitivity  0.74
auc          0.6996191553544495
weights      [0.05, 0.1, 0.25, 0.2, 0.25, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6464285714285715
specificity  0.6400075414781297
sensitivity  0.74
auc          0.6900037707390648
weights      [0.05, 0.1, 0.3, 0.05, 0.1, 0.4]
------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6607142857142857
specificity  0.6533559577677225
sensitivity  0.765
auc          0.7091779788838612
weights      [0.05, 0.1, 0.35, 0.2, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.675
specificity  0.6726244343891403
sensitivity  0.72
auc          0.6963122171945701
weights      [0.05, 0.1, 0.35, 0.2, 0.25, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6571428571428571
specificity  0.6514705882352941
sensitivity  0.74
auc          0.695735294117647
weights      [0.05, 0.1, 0.35, 0.25, 0.05, 0.2]
----------------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6589285714285714
specificity  0.653393665158371
sensitivity  0.735
auc          0.6941968325791855
weights      [0.05, 0.15, 0.3, 0.15, 0.2, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6696428571428571
specificity  0.6668552036199095
sensitivity  0.715
auc          0.6909276018099547
weights      [0.05, 0.15, 0.3, 0.15, 0.25, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.675
specificity  0.6726621417797889
sensitivity  0.715
auc          0.6938310708898944
weights      [0.05, 0.15, 0.3, 0.15, 0.3, 0.05]
--------------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6607142857142857
specificity  0.653393665158371
sensitivity  0.765
auc          0.7091968325791855
weights      [0.05, 0.15, 0.35, 0.2, 0.15, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6714285714285715
specificity  0.6668552036199095
sensitivity  0.74
auc          0.7034276018099547
weights      [0.05, 0.15, 0.35, 0.2, 0.2, 0.05]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6607142857142857
specificity  0.655316742081448
sensitivity  0.74
auc          0.6976583710407239
weights      [0.05, 0.15, 0.35, 0.25, 0.1, 0.1]
----------------------------------------------

...

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6714285714285714
specificity  0.6668174962292609
sensitivity  0.735
auc          0.7009087481146304
weights      [0.3, 0.05, 0.1, 0.25, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6732142857142858
specificity  0.6706259426847663
sensitivity  0.71
auc          0.6903129713423831
weights      [0.3, 0.05, 0.15, 0.05, 0.2, 0.25]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6625
specificity  0.6571644042232277
sensitivity  0.735
auc          0.6960822021116139
weights      [0.3, 0.05, 0.15, 0.15, 0.15, 0.2]
-------------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6732142857142858
specificity  0.6687782805429864
sensitivity  0.74
auc          0.7043891402714932
weights      [0.3, 0.05, 0.35, 0.05, 0.1, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6785714285714286
specificity  0.6765082956259427
sensitivity  0.715
auc          0.6957541478129713
weights      [0.3, 0.05, 0.35, 0.05, 0.15, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6732142857142858
specificity  0.6707013574660633
sensitivity  0.715
auc          0.6928506787330316
weights      [0.3, 0.05, 0.35, 0.1, 0.05, 0.15]
-------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6696428571428571
specificity  0.6629713423831071
sensitivity  0.76
auc          0.7114856711915535
weights      [0.3, 0.1, 0.15, 0.15, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6678571428571428
specificity  0.6610482654600301
sensitivity  0.76
auc          0.710524132730015
weights      [0.3, 0.1, 0.15, 0.2, 0.15, 0.1]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6696428571428572
specificity  0.664894419306184
sensitivity  0.735
auc          0.699947209653092
weights      [0.3, 0.1, 0.2, 0.1, 0.15, 0.15]
---------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6625
specificity  0.6572021116138763
sensitivity  0.735
auc          0.6961010558069382
weights      [0.3, 0.15, 0.05, 0.15, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6642857142857143
specificity  0.6572021116138763
sensitivity  0.76
auc          0.7086010558069381
weights      [0.3, 0.15, 0.05, 0.2, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6642857142857143
specificity  0.6572021116138763
sensitivity  0.76
auc          0.7086010558069382
weights      [0.3, 0.15, 0.1, 0.15, 0.15, 0.15]
--------------------------------------------------------

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6714285714285715
specificity  0.6687405731523379
sensitivity  0.715
auc          0.6918702865761689
weights      [0.35, 0.05, 0.2, 0.05, 0.15, 0.2]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6696428571428571
specificity  0.664894419306184
sensitivity  0.735
auc          0.699947209653092
weights      [0.35, 0.05, 0.2, 0.1, 0.15, 0.15]
----------------------------------------------------------------
----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy     0.6696428571428572
specificity  0.6668174962292609
sensitivity  0.715
auc          0.6909087481146305
weights      [0.35, 0.05, 0.25, 0.05, 0.1, 0.2]
--------------------------------------------

In [16]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.71:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6535714285714286
spes     : 0.6437405731523379
sens     : 0.79
auc      : 0.7168702865761689
weights  : [0.05, 0.05, 0.35, 0.25, 0.1, 0.2]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6785714285714286
spes     : 0.6726244343891403
sens     : 0.76
auc      : 0.7163122171945702
weights  : [0.05, 0.15, 0.35, 0.05, 0.25, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6732142857142858
spes     : 0.6668552036199095
sens     : 0.76
auc      : 0.7134276018099548
weights  : [0.05, 0.15, 0.35, 0.1, 0.2, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6857142857142857
spes     : 0.6822398190045249
sens     : 0.74
auc      : 0.7111199095022624
weights  : [0.05, 0.15, 0.35, 0.1, 0.25, 0.1]

weights  : [0.2, 0.15, 0.15, 0.25, 0.2, 0.05]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6625
spes     : 0.653393665158371
sens     : 0.785
auc      : 0.7191968325791855
weights  : [0.2, 0.15, 0.2, 0.2, 0.1, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6732142857142857
spes     : 0.6649321266968325
sens     : 0.785
auc      : 0.7249660633484163
weights  : [0.2, 0.15, 0.2, 0.2, 0.15, 0.1]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.675
spes     : 0.6687782805429865
sens     : 0.76
auc      : 0.7143891402714933
weights  : [0.2, 0.15, 0.2, 0.2, 0.2, 0.05]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6785714285714286
spes     : 0.6707013574660633
sens     : 0.785
auc      : 

In [17]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.72:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6821428571428572
spes     : 0.6764328808446456
sens     : 0.765
auc      : 0.7207164404223227
weights  : [0.1, 0.05, 0.35, 0.1, 0.25, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6642857142857143
spes     : 0.655316742081448
sens     : 0.785
auc      : 0.720158371040724
weights  : [0.1, 0.1, 0.25, 0.2, 0.2, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6857142857142857
spes     : 0.680316742081448
sens     : 0.765
auc      : 0.722658371040724
weights  : [0.1, 0.1, 0.3, 0.15, 0.25, 0.1]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6910714285714286
spes     : 0.6860859728506787
sens     : 0.765
auc      : 0.7255429864253393
weights  : [0.1, 0.1, 0.35, 0.1, 0.25, 0.1]
-------

In [18]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.73:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6892857142857143
spes     : 0.6822398190045249
sens     : 0.785
auc      : 0.7336199095022624
weights  : [0.2, 0.05, 0.35, 0.1, 0.15, 0.15]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6928571428571428
spes     : 0.6860859728506787
sens     : 0.785
auc      : 0.7355429864253394
weights  : [0.2, 0.1, 0.35, 0.1, 0.15, 0.1]
------------------------------------------------
parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
accuracy : 0.6839285714285714
spes     : 0.676395173453997
sens     : 0.785
auc      : 0.7306975867269985
weights  : [0.25, 0.1, 0.3, 0.15, 0.15, 0.05]
------------------------------------------------
3


In [62]:
#σωστό
params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[3],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.501],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.6], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.8],
                    "reg_alpha"        :[0],
                    "subsample"        :[1.0]
    
        }

In [63]:

weights = [0.2, 0.1, 0.35, 0.1, 0.15, 0.1]

max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)

# parameters : [0.5, 2, 1000, 3, 0.5, 0.5, 1, 3, 0.75, 0, 1.0]
# accuracy : 0.6928571428571428
# spes     : 0.6860859728506787
# sens     : 0.785
# auc      : 0.7355429864253394
# weights  : [0.2, 0.1, 0.35, 0.1, 0.15, 0.1]

----------------------------------------------------------------
[0.5, 2, 1000, 3, 0.501, 0.6, 1, 3, 0.8, 0, 1.0]
accuracy     0.6928571428571428
specificity  0.6860859728506787
sensitivity  0.785
auc          0.7355429864253394
weights      [0.2, 0.1, 0.35, 0.1, 0.15, 0.1]
----------------------------------------------------------------


In [91]:
#σωστό
params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[5],
                    "colsample_bytree" :[1.0],
                    "reg_alpha"        :[0.2],
                    "subsample"        :[1.0]
    
        }
# [3, 11, 13, 14, 15, 1, 12, 2, 10, 5, 8, 6, 0, 4]
# [0.3, 3, 1000, 2, 1, 0.5, 1, 5, 1.0, 0.2, 0.8]
# 0.5875
# 0.5706259426847662
# 0.81
# 0.6903129713423831

In [92]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

In [94]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.65:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

parameters : [0.3, 3, 1000, 2, 1, 0.5, 1, 5, 1.0, 0.2, 1.0]
accuracy : 0.5357142857142857
spes     : 0.5167797888386123
sens     : 0.785
auc      : 0.6508898944193062
weights  : [0.05, 0.25, 0.4, 0.05, 0.2, 0.05]
------------------------------------------------
parameters : [0.3, 3, 1000, 2, 1, 0.5, 1, 5, 1.0, 0.2, 1.0]
accuracy : 0.5357142857142857
spes     : 0.5168174962292609
sens     : 0.785
auc      : 0.6509087481146305
weights  : [0.05, 0.3, 0.35, 0.05, 0.1, 0.15]
------------------------------------------------
parameters : [0.3, 3, 1000, 2, 1, 0.5, 1, 5, 1.0, 0.2, 1.0]
accuracy : 0.5357142857142857
spes     : 0.5167043740573152
sens     : 0.785
auc      : 0.6508521870286577
weights  : [0.1, 0.2, 0.35, 0.05, 0.25, 0.05]
------------------------------------------------
parameters : [0.3, 3, 1000, 2, 1, 0.5, 1, 5, 1.0, 0.2, 1.0]
accuracy : 0.5375
spes     : 0.5187405731523379
sens     : 0.785
auc      : 0.651870286576169
weights  : [0.1, 0.3, 0.35, 0.05, 0.1, 0.1]
----------------

In [31]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.5],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.75], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.2],
                    "subsample"        :[1.0]
    
        }
# parameters : [0.5, 3, 1000, 1, 1, 0.75, 1, 3, 0.5, 0.2, 1.0]
# accuracy : 0.6482142857142857
# spes     : 0.641628959276018
# sens     : 0.74
# auc      : 0.690814479638009
# [13, 2, 3, 15, 11]
# [13, 2, 3, 15, 11, 1, 14, 10, 12, 5, 6, 8, 4, 0, 7, 9, 16]

In [32]:
total= []
for i in range(5,45,5):
    ii = i/100
    for j in range(5,45,5):
        jj = j/100
        for k in range(5,45,5):
            kk = k/100
            for l in range(5,45,5):
                ll = l/100
                for m in range(5,45,5):
                    mm = m/100
                    for n in range(5,45,5):
                        nn = n/100
                        if ii+jj+kk+ll+mm+nn == 1:
                            weights = [ii, jj, kk, ll, mm, nn]
#                             print(weights)
                            max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)
                            total.append([max_sens ,max_params, max_spes, max_acc, max_auc, best_params,weights])

In [37]:
s = 0
for k in range(len(total)):
    for j in range(len(total[k][5])):
        best_params = total[k][5]
        for i in range(len(best_params)):
            if best_params[i][4] > 0.7:
                print("parameters :",best_params[i][0])
                print("accuracy :",best_params[i][1])
                print("spes     :",best_params[i][2])
                print("sens     :",best_params[i][3])
                print("auc      :",best_params[i][4])
                print("weights  :",best_params[i][5])
                print("------------------------------------------------")
                s+=1
print(s)

0


In [26]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3],
                    "colsample_bytree" :[0.5],
                    "reg_alpha"        :[0.1],
                    "subsample"        :[1.0]
    
        }

In [38]:
weights = [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]

max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params,weights)

# parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
# accuracy : 0.6464285714285715
# spes     : 0.6358974358974359
# sens     : 0.79
# auc      : 0.712948717948718
# weights  : [0.25, 0.2, 0.2, 0.1, 0.1, 0.15]

----------------------------------------------------------------
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]
----------------------------------------------------------------


In [ ]:
[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.18, 0.1, 0.1, 0.15]

[0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy     0.6482142857142857
specificity  0.6378205128205128
sensitivity  0.79
auc          0.7139102564102564
weights      [0.27, 0.2, 0.17, 0.1, 0.11, 0.15]

In [ ]:
parameters : [0.5, 3, 1000, 1, 1, 0.5, 1, 3, 0.5, 0, 1.0]
accuracy : 0.6767857142857143
spes     : 0.6725113122171946
sens     : 0.74
auc      : 0.7062556561085973
weights  : [0.05, 0.15, 0.3, 0.2, 0.25, 0.05]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6446428571428571
spes     : 0.6338989441930618
sens     : 0.79
auc      : 0.7119494720965309
weights  : [0.2, 0.25, 0.2, 0.1, 0.15, 0.1]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6464285714285715
spes     : 0.6358974358974359
sens     : 0.79
auc      : 0.712948717948718
weights  : [0.25, 0.2, 0.2, 0.1, 0.1, 0.15]

parameters : [0.4, 3, 1000, 1, 1, 0.5, 1.5, 3, 0.5, 0.1, 1.0]
accuracy : 0.6410714285714285
spes     : 0.6300904977375565
sens     : 0.79
auc      : 0.7100452488687783
weights  : [0.3, 0.25, 0.25, 0.1, 0.05, 0.05]

